In [1]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.cross_validation import KFold

import revrand.basis_functions as bf
from revrand.validation import smse, msll
from revrand.regression import learn, predict
from revrand.btypes import Parameter, Positive

In [2]:
# Load the data
boston = load_boston()
X = boston.data
y = boston.target - boston.target.mean()

N, D = X.shape

In [5]:
# Construct basis functions
base = bf.RandomRBF(Xdim=D, nbases=50, lenscale_init=Parameter(1000*np.ones(D), Positive())) \
    + bf.RandomRBF(Xdim=D, nbases=50, lenscale_init=Parameter(1000, Positive()))

In [6]:
# Cross val
folds = 5
av_smse = 0.
av_msll = 0.
for i, (tr_ind, ts_ind) in enumerate(KFold(len(y), n_folds=folds, shuffle=True)):
    
    # Training
    params = learn(X[tr_ind], y[tr_ind], base)
    
    # Prediction
    Ey, Vf, Vy = predict(X[ts_ind], base, *params)
    
    # Validation
    f_smse = smse(y[ts_ind], Ey)
    f_msll = msll(y[ts_ind], Ey, Vy, y[tr_ind])
    av_smse += f_smse
    av_msll += f_msll
    
    print("Fold: {}, SMSE = {}, MSLL = {}".format(i, f_smse, f_msll))
    
av_smse /= folds
av_msll /= folds

Fold: 0, SMSE = 0.1348670474410108, MSLL = -1.121755601718169
Fold: 1, SMSE = 0.12502128230768073, MSLL = -1.1045212809749994
Fold: 2, SMSE = 0.35207851560968784, MSLL = -0.30482597967425873
Fold: 3, SMSE = 0.16490815289525493, MSLL = -0.9426245569401307
Fold: 4, SMSE = 0.16450563576183283, MSLL = -0.8372288685995053


In [7]:
# Print results
print("Final: SMSE = {}, MSLL = {}".format(av_smse, av_msll))

Final: SMSE = 0.1882761268030934, MSLL = -0.8621912575814126
